# Hey Pentti, We Did It! A Fully Vector-Symbolic Lisp

The following is code for a vector-symbolic implementation of a Lisp interpreter. The interpreter is admittedly slow and inefficient and the Lisp is restricted to the minimal operators sufficient for Turing-completeness. But the code constitutes a proof-of-concept that Lisp can be implemented using a vector-symbolic architecture.

The interpreter is described formally in the following paper. If you are looking to cite this code, cite this paper:

> Tomkins-Flanagan, E. & Kelly, M. A. (2024). Hey Pentti, we did it!: A fully vector-symbolic Lisp. In *Proceedings of the 22nd International Confernece on Cognitive Modeling*. Tilburg, the Netherlands: Tilburg University.

The code is written by E. Tomkins-Flanagan and M. A. Kelly by modifying the minimal Lisp interpreter written in Python by Peter Norvig.

The original code by Norvig can be found here:

https://www.norvig.com/lispy.html

## HRR classes

Here we define a class for Tony Plate's Holographic Reduced Representations (HRRs; Plate, 1995), a type of Vector Symbolic Architecture (VSA), as well as HRRscale, which is for enforcing lazy evaluation by not evaluating functions when the function result is multiplied by a value close to zero.

In [ ]:
#@title Holographic Reduced Representations

import numpy as np # import linera algebra library
from numpy.fft import fft,ifft
import math # you also need to import basic math because python is silly
from collections.abc import Sequence

# find the inverse permutation given a permutation
def invPerm(perm):
    inv       = np.arange(perm.size) # initialize inv
    inv[perm] = np.arange(perm.size) # create inv from perm
    return inv

# used exclusively for handling coefficients in a conditional expression;
# if multiplying a function, does *not* evaluate that function when scale is
# close to 0, preventing undefined behaviour in cases that are not supposed
# to be reachable (presumably, a more efficient implementation of conditional
# evaluation could be written)
class HRRscale:
    def __init__(self, scale, value=None, large=0.8, small=0.2, env=None):
        self.large, self.small = large, small
        self.scale = scale
        self.value = value
        if isinstance(value, HRR):
            self.value = lambda: value
        elif isinstance(value, np.ndarray):
            if env:
                v = lispexpr(data=value, env=env)
            else:
                v = HRR(data=value)
            self.value = lambda: v
        elif hasattr(value, '__iter__'):
            fname, *args = value
            self.value = lambda: fname(*args)
        elif callable(value):
            self.value = value

    def __mul__(self, other):
        if isinstance(other, float):
            if isinstance(other, HRRscale):
                if other.value is not None and self.value is not None:
                    raise ValueError("Two HRRscales cannot multiply if both are unevaluated. Evaluate one first.")
                elif other.value is not None:
                    return HRRscale(self.scale*other.scale, value=other.value)
                else:
                    return HRRscale(self.scale*other.scale, value=self.value)

            return HRRscale(self.scale*other, value=self.value)

        # catches the edge case where we just multiply an unevaluated lambda with no arguments (I don't know when we'll be doing this but we prob shouldn't be)
        elif isinstance(other, HRR) or callable(other):
            if self.value is not None:
                raise ValueError(f"HRRscale is unevaluated and contains a value! Evaluate it first before multiplying it with another object.\n({self.scale}, {self.value})")
            return HRRscale(scale=self.scale, value=other)

        # the lazy magic, we provide "other" as a tuple with the name of the function and the sequence of arguments
        elif hasattr(other, '__iter__'):
            if self.value is not None:
                raise ValueError(f"HRRscale is unevaluated and contains a value! Evaluate it first before multiplying it with another object.\n({self.scale}, {self.value})")

            fname, *args = other
            return HRRscale(scale=self.scale, value=lambda: fname(*args))

        raise ValueError("HRRscales can only multiply floats, HRRscales, HRRs, functions, decomposed functions")

    def __call__(self):
        if callable(self.value):
            self.value = self.value()
        return (self.scale > self.large or self.scale) * self.value

    def __or__(self, other):
        if self.scale > self.large:
            return self()
        elif self.scale < self.small:
            return other
        else:
            return self() + other

    def __add__(self, other):
        return self() + other

    def __repr__(self):
        return f'HRRscale({repr(self.scale)} * {repr(self.value)})'


# class for Tony Plate's Holographic Reduced Representations
class HRR(Sequence):
    # Generate a vector of values sampled from a normal distribution
    # with a mean of zero and a standard deviation of 1/N
    def __init__(self,N=None,data=None, zero=False,large=0.8,small=0.2):
        self.large = large
        self.small = small
        if data is not None: # the vector is specified rather than random
            if isinstance(data, HRR):
                self.v = data.v
            else:
                self.v=np.array(data,dtype=float)
        elif zero and N is not None:
            self.v = np.zeros(N)
        elif N is not None: # create a random vector of N dims
            sd=1.0/math.sqrt(N)
            self.v=np.random.normal(scale=sd, size=N)
            self.v/=np.linalg.norm(self.v)
        else:
            raise Exception('Must specify size or data for HRR')

        self.scale = np.linalg.norm(self.v)

    # The multiplication-like operation is used to associate or bind vectors.
    def __mul__(self,other):
        if isinstance(other,HRR):
            return HRR(data=ifft(fft(self.v)*fft(other.v)).real)
        else:
            return HRR(data=self.v*other)

    def __rmul__(self,other):
        return self * other

    # exponentiation is used for fractional binding
    def __pow__(self,exponent):
        x=ifft(fft(self.v)**exponent).real
        return HRR(data=x)

    # The addition-like operation is used to superpose vectors or add them to a set.
    def __add__(self,other):
        if isinstance(other,HRR) or isinstance(other, HRRscale):
            return other + self.v
        else:
            return HRR(data=other + self.v)

    # allows us to specify the negative of a vector, -HRR
    def __neg__(self):
        return HRR(data=-self.v)

    # allows subtracting HRR from HRR
    def __sub__(self,other):
        return HRR(data=self.v-other.v)

    # An inverse can be defined such that binding with the inverse unbinds
    def __invert__(self):
        return HRR(data=self.v[np.r_[0,self.v.size-1:0:-1]])

    # Unbinding approximately cancels out binding
    # Unbinding in HRRs is implemented as the binding of the inverse of the cue to the trace.
    def __truediv__(self,other):
        if isinstance(other,HRR):
            return self * ~other
        else: # actually just divide if the divisor is not an HRR
            return HRR(data=self.v/other)

    # retrieve the dimensionality of the vector
    def __len__(self):
        return HRR.v.size

    # index into the vector
    def __getitem__(self, i):
        if isinstance(i,int):
            return self.v[i]
        else:
            return HRR(data=self.v[i])

    def __setitem__(self, key, value):
        self.v[key] = value

    # get the Euclidean length or magnitude of the vector
    # use self.scale most of the time, as it's pre-calculated
    def magnitude(self):
        return math.sqrt(self.v @ self.v)

    # compare two vectors using the vector cosine to measure similarity
    def __eq__(self,other):
        scale = self.scale * other.scale # scaling for normalization
        if scale==0:
            return 0
        return (self.v @ other.v)/scale

    # normalize to the unit vector, i.e., a magnitude or Euclidean length of 1
    def unit(self):
        return HRR(data=self.v / self.scale)

    # dimensionality of our vector-symbol
    # added by Eilene
    def size(self):
        return self.v.size

    # dimensionality of our vector-symbol
    def shape(self):
        return self.v.shape

    # typically the vector dot product
    # operator @
    # added by Eilene
    def __matmul__(self, other):
        if isinstance(other, HRR):
            return self.v @ other.v
        elif isinstance(other, np.ndarray) and len(other.shape) == 2:
            return HRR(data=self.v @ other)
        else:
            return self.v @ other

    # projection of self onto other
    # used in Widdow's PSI model
    # and quantum models of decision making
    def proj(self,other):
        return (self @ other.unit()).item()

    # implements PSI negation
    # returns self "without" the other
    # as self minus the projection of self onto onther
    def reject(self,other):
        return self - (self.proj(other))*other

    # created by Eilene
    # a variant addition operator
    # "a | b" is a if a has a large magnitude
    # otherwise it's b if a has a small magnitude
    # lastly, it could be both a and b if a has a middling magnitude
    def __or__(self,other):
        if self.scale > self.large:
            return self
        elif self.scale < self.small:
            return other
        else:
            return self+other

    # print HRR vector
    def __repr__(self):
        return f'HRR({self.v})'

    # for using an HRR as a key in a hash table
    def __key(self):
        return (*self.v,)

    # for using an HRR as a key in a hash table
    def __hash__(self):
        return hash(self.__key())

# Wraps HRRs with a reference to an env and some additional logic such that they can be
# interpreted as lisp expressions using the memory object in the env
# Used mainly for __repr__, which calls read to translate the object to a human-readable
# expression, O(n*g(m)) where g is the lookup time as a function of memory size m, and
# n is the number of atoms in the expression
class lispexpr(HRR):
    def __init__(self,N=None,data=None, zero=False,large=0.8,small=0.2,env=None):
        super().__init__(N=None,data=None, zero=False,large=0.8,small=0.2)
        if env:
            self.env = env
        else:
            raise ValueError('Keyword argument env required')

    # Evaluates the expression
    def __call__(self):
        return self.env['evall'](self)

    # Iteration only works for lists
    def __iter__(self):
        return self

    # Basically unused, allows us to iterate over lispexprs
    def __next__(self):
        if (self==env['NIL']) > self.small: # or (self==closest(self, env['lex'])) < self.small: # this makes iteration work for tuples but it's slow as shit
            raise StopIteration
        head = env['CAR'](self)
        tail = env['CDR'](self)
        self.v, self.scale = tail.v, tail.scale
        return head

    # Calls read to return a human-readable rendition of the expression, together with its
    # vector representation
    def __repr__(self):
        return repr(self.env['read'](self)) + ' : ' + super().__repr__()

    # Evaluates then compares expressions, == is expected to be a cosine similarity
    def __eq__(self, other):
        return self() == other() if isinstance(other, lispexpr) else self() == other

    def __add__(self, other):
        return lispexpr(data=super().__add__(other), env=self.env)

    def __sub__(self, other):
        return lispexpr(data=super().__sub__(other), env=self.env)

    def __invert__(self):
        return lispexpr(data=super().__invert__(), env=self.env)

    def __mul__(self, other):
        return lispexpr(data=super().__mul__(other), env=self.env)

    def __matmul__(self, other):
        return lispexpr(data=super().__matmul__(other), env=self.env)

## Lisp environment

Below, we specify an environment that defines the Lisp functions used in our paper. Due to certain implementation requirements (and, to be fair, questionable choices on my part), the environment includes the complete interpreter, so Norvig's interpreter is refactored into a thin wrapper for environment calls. The environment is a dictionary, generated by a function, rather than a class for some reason. Probably because that's how Norvig did it.

We note that execution is *extremely* slow and memory-intensive. What we have produced is a proof-of-concept, not something that can be dumped without modification into one's models.

In [ ]:
#@title Lisp environment definitions

from ast import Expression
from collections import UserDict

import traceback

class SimpleCleanUp():
    def __init__(self,n=512,m=100):
        self.M = np.zeros((m,n)) # memory matrix
        self.n = n # trace dimensionality
        self.m = m # initial maximum number of traces
        self.k = m # increment for adding more traces
        self.i = 0 # current number of traces

    def memorize(self,hrr):
        if self.i >= self.m:
            self.M = np.concatenate([self.M,np.zeros((self.k,self.n))],axis=0)
        self.M[self.i,:]= hrr.v
        self.i=self.i+1
        return hrr

    def recall(self,hrr):
        activations = self.M @ hrr.v
        return HRR(data=self.M[np.argmax(activations),:])

# An object to contain all the base symbols in both their string and
# holographic vector representations
# A string s can be looked up, given an HRR h, for lexicon L, with L.reverse[h]
# Similarly, L[s] = h
class Lexicon(UserDict):
    def __init__(self, mapping=None, /, **kwargs):
        self.reverse = {}
        super().__init__(mapping, **kwargs)

    def __setitem__(self, key, value):
        super().__setitem__(key, value)
        self.reverse[value] = key

    def update(self, other=(), /, **kwds):
        super().update(other, **kwds)
        self.reverse.update(other=(((v, k) for k, v in other),))
        self.reverse.update(other=(((v, k) for k, v in kwds.items()),))

# A heteroassociative memory object employed because tuples turn out to degrade
# rather quickly. When CONS is called, we return a pointer to the tuple in the
# associative memory, which is automatically dereferenced by CAR, CDR
# Attention must still be paid to the action of the pointer, as it is an HRR
# and it is uncorrelated with its associate, and using it rather than
# dereferencing it can lead to errors
class SimpleAssoc():
    def __init__(self, lexicon=Lexicon(), theta=0.2):
        self.A = Lexicon()
        self.lexicon = lexicon
        self.reverse = lexicon.reverse
        self.theta = theta

    def memorize(self, probe, trace):
        if trace not in self.A.reverse:
            self.A[probe] = trace
        else:
            probe = self.A.reverse[trace]

        return (probe, trace)

    def recall(self, probe):
        if probe in self.A:
            return self.A[probe]

        keys = list(self.A.keys())
        activations = [probe==HRR(data=k) for k in keys]
        nearest = np.argmax(activations)

        if (activations[nearest]) > self.theta:
            return self.A[keys[nearest]]
        else:
            return probe

# find the closest vector in the lexicon dictionary and return its name/key
def closest(target,lexicon,exclusions=[]):
    max_sim = 0
    max_word= "NONE"
    for word in lexicon:
        if word not in exclusions and isinstance(lexicon[word],HRR):
            sim = lexicon[word] == target
            if sim > max_sim:
                max_sim = sim
                max_word= word
    return max_word


# find the top 5 closest to the target
def top5(target,lexicon):
    word_list = list(lexicon)
    word_list.sort(reverse=True,key=lambda x: lexicon[x]==target if isinstance(lexicon[x], HRR) else 0.)
    return word_list[:5]

# Calling this function yields a dictionary containing the Lisp environment,
# called env
def vsa_env(dim=512):
    "An environment that defines LISP as vector algebra and Hopfield nets."
    env   = {'lex': Lexicon()}
    mem   = SimpleCleanUp(dim)
    func  = SimpleCleanUp(dim)
    assoc = SimpleAssoc(lexicon=env['lex'])

    floor = 0.2

    # define reserved terms
    reserved  = ['T','F',"NIL",'lhs','rhs','φ',1]
    for word in reserved:
        env[word] = HRR(N=dim)
        env['lex'][word] = env[word]
        mem.memorize(env[word])

    # define basic functions
    basic_fs  = ['CAR', 'CDR', 'CONS', 'EQ', 'ATOM', 'QUOTE', 'DEFINE', 'COND', 'LAMBDA', 'LABEL']
    for f in basic_fs:
        env['lex'][f] = HRR(N=dim)
    env['basic'] = basic_fs

    # function definitions
    def quote(c, ind=''):   return make_list([env['lex']['QUOTE'], c])
    def cons(ab, ind=''):   return ccons(car(ab), car(cdr(ab)))
    def car(c, ind=''):     return mem.recall(assoc.recall(c) * ~env["lhs"])
    def cdr(c, ind=''):     return mem.recall(assoc.recall(c) * ~env["rhs"])
    def eq(ab, ind=''):     return ((a := (ev(car(ab), ind=ind+'  ')))==ev(b := ev(car(cdr(ab), ind=ind+'  '), ind=ind+'  ')))*env['T'] + (1 - (a==b))*env["F"]
    def atomic(a, ind=''):  return mem.recall(((aa := assoc.recall(a))==env['φ'])*env['F'] + max((2 * floor) - (aa==env['φ']), 0.)*env["T"])
    def atom(ab, ind=''):   return ((a := cdr(ab))==env['NIL']) * atomic(car(ab)) +  max((2*floor) - (a == env['NIL']), 0.) * env['F']
    #def cond(r, ind='', br=1):    return HRRscale((rr := ev_until_done(car(car(r)), ind=ind+'  '))==env["T"]) * (lambda: (print(f'{ind}branch {br} taken because of test condition {read(rr)}: {read(r)}'), cdr(car(r)))[1]) | HRRscale(1.) * (lambda: (print(f'{ind}branch {br} skipped because of test condition {read(rr)}: {read(r)}'), cond(cdr(r), ind=ind+'  ', br=br+1))[1]) | False
    def cond(r, ind=''):    return HRRscale(ev_until_done(car(car(r)), ind=ind+'  ')==env["T"]) * (lambda: cdr(car(r))) | HRRscale(1.) * (lambda: cond(cdr(r), ind=ind+'  ')) | False

    # Puts a name-lambda association in the function memory
    def define(name_exp, ind=''):
        #print('define:', read(name_exp))
        n, e = car(name_exp), car(cdr(name_exp))
        trace = ccons(n, e)
        func.memorize(assoc.recall(ccons(n, e))) # note: this means that function definitions won't always be uncorrelated with other vectors
        #print(assoc.recall(trace) == trace)
        #print(assoc.recall(trace), trace)
        #i = 0
        #while func.M[i].any():
        #    i += 1
        #print(func.M[i-1])
        #print('defined:', read(func.recall(name_exp)))
        return env['NIL']

    # These are relabeled CAR and CDR in the env namespace
    # When you write a lisp program and call (CAR (a . b)), the tail of the
    # expression is ((a . b)), so we always need to pull out (a . b), and then
    # run CAR over it
    def caar(c, ind=''): return car(car(c))
    def cdar(c, ind=''): return cdr(car(c))

    # In this case, cons is the version that interacts with the Lisp interpreter, and
    # ccons is the under-the-hood version; don't ask me why I did this
    def ccons(a,b, ind=''):
        pointer = HRR(dim)
        mem.memorize(a)
        mem.memorize(b)
        pointer, trace = assoc.memorize(pointer, a*env["lhs"] + b*env["rhs"] + env['φ'])
        return pointer

    # helper function for label
    # takes a label l and two expressions d and e
    def ll(l,d,e):
        return ccons(HRRscale(1-(car(d)==l))*car(d)
                    |HRRscale(car(d)==l)*e
                    |False,
                    HRRscale(cdr(d)==env["NIL"])*env["NIL"]
                    |HRRscale(1-(cdr(d)==env['NIL']))*(lambda: ll(l,cdr(d),e))
                    |False)

    # recursively replaces instances of label in expression
    def label(lbl_exp):
        lbl, exp = car(lbl_exp), car(cdr(lbl_exp))
        mem.memorize(lbl,exp)
        return ll(lbl,exp,exp)

    # lexpr is the lambda expression
    # x is the list of parameters
    # e is the expression to be evaluated by substituting parameters for argument
    # a is the argument that the lambda function is called with
    # all lambda expressions are curried, so you can't pass in a list of arguments

    # Top-level lambda evaluator; this is what you call to evaluate a lambda expression
    def evlamb(lexpr, a):
        x, e = (None, None)
        if isinstance(lexpr, HRR):
            tail = cdr(lexpr)                 # list of form (LAMBDA x e), we retrieve (x e)
            x, e = car(tail), car(cdr(tail))  # we want e = (x y z ...) and not ((x y z ...) NIL)
        elif isinstance(lexpr, tuple):
            x, e = lexpr
        else:
            raise ValueError(f'''Error evaluating lambda expression (function evlamb): expected either HRR list or 2-tuple of HRR lists for type of parameter lexpr, got {type(lexpr)}.
Use the traceback to see what is being passed to this function.
String rep of lexpr: {str(lexpr)}
Value of lexpr: {lexpr}''')

        return (HRRscale(x==env["NIL"]) * e                                     # base case 1: there are no parameters left, so return the inner expression
                | HRRscale(a==env['NIL']) * (lambda: mklamb(x, e))              # base case 2: there are no arguments left, so return the identity
                | HRRscale(1.) * (lambda: evlamb(evlambcurry((x, e), make_list([car(a)])), cdr(a))) # recursive case: curry the arguments and evaluate the lambda on each
                | False
        )

    # Second-level lambda evaluator; works on curried expressions, called by evlamb
    def evlambcurry(lexpr,a):
        x, e = (None, None)
        if isinstance(lexpr, HRR):
            tail = cdr(lexpr)                 # list of form (LAMBDA x e), we retrieve (x e)
            x, e = car(tail), car(cdr(tail))  # we want e = (x y z ...) and not ((x y z ...) NIL)
        elif isinstance(lexpr, tuple):
            x, e = lexpr
        else:
            raise ValueError(f'''Error evaluating lambda expression (function evlambcurry): expected either HRR list or 2-tuple of HRR lists for type of parameter lexpr, got {type(lexpr)}.
Use the traceback to see what is being passed to this function.
String rep of lexpr: {str(lexpr)}
Value of lexpr: {lexpr}''')
        return (HRRscale(x==env["NIL"]) * (lambda: e)                           # base case 1: there are no more parameters, so return the inner expression
                | HRRscale(e==env["NIL"]) * (lambda: env["NIL"])                # base case 2: we have reached the end of the inner expression, there's nothing left to evaluate
                | HRRscale(1.) * (lambda: mklamb(cdr(x), evshorn((x, e), a)))   # recursive case *: in any other case, we need to evaluate the lambda expression,
                | False                                                         #                   and return a lambda expression on one fewer parameter containing the result
                )

    # Bottom-level lambda evaluator; recursively constructs the tail of a beta-reduction
    # This is separated from evlambcurry because lambcurry needs to return a lambda expression if parameters remain after evaluation
    # but we don't want it putting a bunch of lambda expressions in the tail during evaluation
    def evshorn(lexpr, a):
        x, e = (None, None)
        if isinstance(lexpr, HRR):
            tail = cdr(lexpr) # list of form (LAMBDA x e), we retrieve (x e)
            x, e = car(tail), car(cdr(tail)) # we want e = (x y z ...) and not ((x y z ...) NIL)
        elif isinstance(lexpr, tuple):
            x, e = lexpr
        else:
            raise ValueError(f'''Error evaluating lambda expression (function evshorn): expected either HRR list or 2-tuple of HRR lists for type of parameter lexpr, got {type(lexpr)}.
Use the traceback to see what is being passed to this function.
String rep of lexpr: {str(lexpr)}
Value of lexpr: {lexpr}''')
        return (HRRscale(x==env["NIL"]) * (lambda: e)                           # base case 1: there are no parameters, so return the inner expression
                | HRRscale(e==env["NIL"]) * (lambda: env["NIL"])                # base case 2: we have reached the end of the inner expression, there's nothing left to evaluate
                | HRRscale(car(x)==e) * (lambda: car(a))                        # base case 3: we are at the end of a tuple and we need to do a substitution
                | HRRscale(atomic(e)==env['T']) * (lambda: e)                   # base case 4: we are at the end of a tuple, there's nothing left to evaluate
                | HRRscale(car(x)==car(e)) * (lambda: ccons(car(a),evshorn((x,cdr(e)), a)))                             # recusrive case 1: the head of the list of parameters equals the head of the inner expression, so substitute the argument for the head of e
                | HRRscale(atomic(car(e))==env["F"]) * (lambda: ccons(evshorn((x,car(e)),a), evshorn((x,cdr(e)), a)))   # recursive case 2: the head of the list of parameters is nonatomic, so we must recurse both on it and the tail
                | HRRscale(1.) * (lambda: ccons(car(e), evshorn((x,cdr(e)),a)))                                         # recursive case 3: the head of the list of parameters is atomic, but not equal to the head of the inner expression, so we leave the head of e and recurse on the tail
                | False
                )

    # constructs a lambda expression
    def mklamb(x, e):
        return make_list([env['lex']['LAMBDA'], x, e])

    # add a new atomic expression to memory
    def make_atom(name):
        if name not in env['lex']:
            env['lex'][name] = mem.memorize(HRR(dim))
        return env['lex'][name]

    # make a list vector
    # a is a list, not a vector
    def make_list(a):
        if not a:
            return env["NIL"]

        head, *tail = a

        head = to_HRR(head)

        if not tail:
            return ccons(head, env["NIL"])
        else:
            return ccons(head, make_list(tail))

    # Turns lists, tuples, strings, into HRRs
    def to_HRR(a):
        if isinstance(a, HRR):
            return a
        elif isinstance(a, str):
            return make_atom(a)
        elif isinstance(a, list):
            return make_list(a)
        elif isinstance(a, tuple):
            return make_tuple(a)

        raise TypeError('function to_HRR only handles strings, 2-tuples, lists, HRRs')


    # The only tuples are 2-tuples
    def make_tuple(a):
        if len(a) != 2:
            raise ValueError('Only 2-tuples may be constructed!')

        left, right = a
        left, right = to_HRR(left), to_HRR(right)

        return ccons(left, right)

    # Turn an HRR tuple back into a human-readable representation
    # May not return exactly the representation you put in as a tuple containing
    # a list of length n is exactly equivalent to a list of length n+1
    # and 'NIL' is identical with []
    def read(v):
        #print(v, 'atomic?', atomic(v)==env['T'])
        if (atomic(v)==env['T']) > floor:
            return closest(mem.recall(v), env['lex'])

        left, right = read(car(v)), read(cdr(v))

        if isinstance(right, list):
            return [left, *right]
        elif right == 'NIL':
            return [left]
        else:
            return (left, right)

    # Evaluates an expression
    # Could be defined in terms of the basic functions; it would add overhead
    # The evaluation may return another expression that can be evaluated again
    def ev(e, ind=''):
        #traceback.print_stack()

        # If e is atomic, leave it alone (base case)
        if (atomic(e)==env['T']) > floor:
            #print(f'{ind}{read(e)} is atomic')
            return e

        head, tail = car(e), cdr(e)

        #st_head, st_tail = read(head), read(tail)

        # If head is lambda, this is a lambda expression, we need to leave it for now
        if (head==env['lex']['LAMBDA']) > floor:
            #print(f'{ind}{st_head} is lambda, so we are in a lambda expression; it will remain a literal until called')
            return e

        # If head is a basic function, call the basic function
        # EXCEPT lambda. DO NOT put LAMBDA in env, this is handled separately
        if head in env['lex'].reverse and env['lex'].reverse[head] in basic_fs: # env['basic']

            #print(f'{ind}{st_head} is the name of a basic function to operate on {st_tail}')
            #print(f'{ind}{read(head)} is the name of a basic function to operate on {read(tail)}')
            #print(f'{ind}{env[env["lex"].reverse[head]]}')
            result = env[env['lex'].reverse[head]](tail, ind=ind+'  ')
            #print(f'{ind}{read(result)} was returned') # {result}')
            return result

        # If head is a defined function, retrieve the function definition
        fprobe = env['lhs'] * head
        echo = func.recall(fprobe)
        if (car(echo)==head) > floor:
            #print(f'{ind}{st_head} is a programmer-defined function to operate on {st_tail} after substitution for bound lambda expression')
            head = cdr(echo)
            #st_head = read(head)

        # If head is a lambda expression, evaluate it
        if (car(head)==env['lex']['LAMBDA']) > floor:
            #print(f'{ind}{st_head} is a lambda expression to operate on {st_tail}')
            #return evlamb(head, tail) # this is how it was at last run, I think I need to evaluate the arguments (tail) first, but I think doing so breaks things
            tail = ev_until_done(tail, ind=ind+'  ')
            #st_tail = read(tail)
            #print(f'{ind}After evaluation, tail is {st_tail}')
            return evlamb(head, tail)

        #print(f'{ind}{st_head} is unrecognized with tail {st_tail}')
        # otherwise, just leave it and evaluate the tail
        return ccons(ev(head, ind=ind+'  '), ev(tail, ind=ind+'  '))

    # Evaluates an expression, and if the result is evaluable, evaluates it again
    # This is tail-recursive so Python will interpret it as a loop
    def ev_until_done(e, ind=''):
        #print(f'{ind}evaluating {read(e)}')
        e, ee = ev(e, ind=ind+'  '), e
        #print(f'{ind}done with e = {read(e)}')
        head = car(e)
        # if e is an atom, a quote, or some other literal expression
        # (like something that evaluates to itself), just return e
        if (
              (atomic(e)==env['T']) > floor or
              (head == env['lex']['QUOTE']) > floor or
              (head==env['lex']['LAMBDA']) > floor or
              (ee==e) > floor
            ):
            #print(f'{ind}{read(e)} is atom, lambda expression, or literal')
            return e
        # if e's head is atomic, but we don't know how to evaluate it, leave it and evaluate the tail
        elif (
                (atomic(head)==env['T']) > floor and
                (car(func.recall(head * env['lhs']))==head) < floor and
                (
                  head not in env['lex'].reverse or
                  env['lex'].reverse[head] not in env['basic']
                )
              ):
            #print(f'{ind}{read(head)} is atomic but is not a known function')
            return ccons(head, ev_until_done(cdr(e), ind=ind))

        # otherwise, keep going
        #print(f'{ind}{read(e)} may still be evaluated')
        return ev_until_done(e, ind=ind)

    env.update({
        'CAR':          caar,
        'CDR':          cdar,
        'CONS':         cons,
        'EQ':           eq,
        'ATOM':         atom,
        'QUOTE':        quote,
        'COND':         cond,
        'DEFINE':       define,
        'LABEL':        label,
        '(LAMBDA x e)': evlamb,
        'mem' :         mem,
        'func':         func,
        'assoc':        assoc,
        'holo':         to_HRR,
        'read':         read,
        'ev':           ev,
        'evall':        ev_until_done,
    })

    return env

## Lispy interpreter

In [ ]:
# @title Lispy: Scheme Interpreter in Python by Peter Norvig

## (c) Peter Norvig, 2010-16; See http://norvig.com/lispy.html
## Modified by Eilene Tomkins-Flanagan and Mary Alexandria Kelly, 2024

################ Parsing: parse, tokenize, and read_from_tokens

def parse(program):
    "Read a Scheme expression from a string."
    parsed_program = []
    tokens = tokenize(program)
    # read_expr_from_tokens consumes one top-level expression so we just iterate until we run out
    while tokens:
        parsed_program.append(read_expr_from_tokens(tokens))

    return parsed_program

def tokenize(s):
    "Convert a string into a list of tokens."
    return s.replace('(',' ( ').replace(')',' ) ').split()

def read_expr_from_tokens(tokens):
    "Read an expression from a sequence of tokens."
    if len(tokens) == 0:
        raise SyntaxError('unexpected EOF while reading')
    token = tokens.pop(0)
    if '(' == token:
        L = []
        while tokens[0] != ')':
            L.append(read_expr_from_tokens(tokens))

        tokens.pop(0) # pop off ')'

        if len(L) == 3 and L[1] == '.':
          return (L[0], L[2])

        return L
    elif ')' == token:
        raise SyntaxError('unexpected )')
    elif token.isnumeric():
        return float(token)
    else:
        return str(token)

################ Environments

global_env = vsa_env()

################ Interaction: A REPL (read-evaluate-print loop)

def repl(prompt='holis.py> '):
    "A prompt-read-evaluate-print loop."
    while True:
        txt = input(prompt)
        if txt in ["", "exit","quit","exit()","quit()","(quit)","(exit)"]:
            print("Goodbye!")
            break
        run(txt)

def lispstr(exp):
    "Convert a Python object back into a Lisp-readable string."
    return global_env['read'](exp)

def run(txt):
    vals = lisp(*parse(txt))
    if vals is not None:
        for val in vals:
            print(lispstr(val))


################ Interpret

def lisp(*x, env=global_env):
    "Evaluate an expression in an environment."
    return [env['evall'](env['holo'](s)) for s in x]

## Example Program

Below is an example fizzbuzz program. The ultimate call is omitted, and can be performed by typing `(fizzybuzzy i)` into repl. `i` is a natural number (including 0), defined as `0 = NIL`, and `n + 1 = (n)`, for all natural numbers `n > 0`.

In [ ]:
# @title fizzbuzz

# The function fizzesbuzzes will give you fizzbuzz in reverse order because it was simpler that way
# fizzingbuzzing does it in the correct order but it's lazy evaluated and goes to infinity from a defined start point
# Finally fizzybuzzy takes n from fizzingbuzzing
# Fizzybuzzy most closely resembles the behaviour of a normal fizzbuzz program

# As we don't represent numbers, this program uses NIL to represent 0, and each
# subsequent natural number is taken as the list containing only the previous natural number, so
# 1 := (0) = (NIL)
# 2 := (1) = ((NIL))
# etc.

lisp_fizzbuzz_program = """
(DEFINE and
  (LAMBDA
    (a b)
    (COND
      ((EQ a T) . (COND ((EQ b T) . T) (T . F)) )
      (T . F)
    )
  )
)

(DEFINE take
  (LAMBDA
    (n l)
    (COND
      ((EQ n NIL) . NIL)
      (T .
        (CONS
          (CAR l)
          (take (CAR n) (CDR l))
        )
      )
    )
  )
)

(DEFINE countdown
  (LAMBDA
    (x m)
    (COND
      ((EQ m NIL) . x)
      ((EQ x NIL) . F)
      (T . (countdown (CAR x) (CAR m)) )
    )
  )
)

(DEFINE divisible
  (LAMBDA
    (x n)
    ((LAMBDA
      (xx)
      (COND ((EQ xx F) . F) ((EQ xx NIL) . T) (T . (divisible xx n) ) )
    ) (countdown x n) )
  )
)

(DEFINE fb
  (LAMBDA
    (i)
    (COND
      (((and ( (divisible i) (((NIL))) )) ( (divisible i) (((((NIL))))) )) . (QUOTE fizzbuzz))
      (( (divisible i) (((NIL))) ) . (QUOTE fizz))
      (( (divisible i) (((((NIL))))) ) . (QUOTE buzz))
      (T . i)
    )
  )
)

(DEFINE fbfb
  (LAMBDA
    (i)
    (COND
      ( ( and ( divisible i (((NIL))) ) ( divisible i (((((NIL))))) ) ) . (QUOTE fizzbuzz))
      (( divisible i (((NIL))) ) . (QUOTE fizz))
      (( divisible i (((((NIL))))) ) . (QUOTE buzz))
      (T . i)
    )
  )
)

(DEFINE fizzesbuzzes
  (LAMBDA
    (n)
    (COND
      ( (EQ n (NIL)) . ( (NIL) ) )
      (T . ( (fb n) . (fizzesbuzzes (CAR n)) ))
    )
  )
)

(DEFINE fizzingbuzzing
  (LAMBDA
    (i n)
    (COND
      ((EQ i n) . ((fb i)))
      (T . ( (fb i) . (fizzingbuzzing (i) n) ))
    )
  )
)

(DEFINE fizzybuzzy
  (LAMBDA
    (n)
    ( fizzingbuzzing (NIL) n )
  )
)
"""

run(lisp_fizzbuzz_program)

# Interactive environment

In [ ]:
repl()